In [ ]:
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
from scipy import signal
from scipy import stats


Drive already mounted at drive/; to attempt to forcibly remount, call drive.mount("drive/", force_remount=True).


# User 46


In [ ]:
st0 = time.time()
user46 = pd.read_csv("raw_field_acc_user46.csv")
st1 = time.time()
user46


,user_id,datetime,x,y,z
0,27,2018-12-11T16:46:05.707+0900,-0.459,1.072,9.730
1,27,2018-12-21T16:45:54.350+0900,-1.532,-9.577,0.043
2,27,2018-12-21T10:38:58.230+0900,0.996,1.225,9.683
3,27,2018-12-14T22:39:10.469+0900,-0.229,0.766,9.773
4,27,2018-12-07T15:43:06.481+0900,-1.953,1.110,9.543
...,...,...,...,...,...
12539426,27,2018-12-21T00:03:21.190+0900,8.964,-0.498,-0.842
12539427,27,2018-12-21T02:27:55.440+0900,8.925,-0.498,-0.874
12539428,27,2018-12-21T01:47:50.623+0900,9.002,-0.498,-0.852
12539429,27,2018-12-21T00:31:11.222+0900,9.002,-0.498,-0.991


In [ ]:
st = time.time()
user46.datetime = pd.to_datetime(user46.datetime).dt.tz_localize(None)
et = time.time()
print("Datetime conversion time:", et - st)
print(user46.dtypes)
print(user46.head(10))


Datetime conversion time: 198.39138650894165
user_id              int64
datetime    datetime64[ns]
x                  float64
y                  float64
z                  float64
dtype: object
   user_id                datetime      x      y      z
0       27 2018-12-11 16:46:05.707 -0.459  1.072  9.730
1       27 2018-12-21 16:45:54.350 -1.532 -9.577  0.043
2       27 2018-12-21 10:38:58.230  0.996  1.225  9.683
3       27 2018-12-14 22:39:10.469 -0.229  0.766  9.773
4       27 2018-12-07 15:43:06.481 -1.953  1.110  9.543
5       27 2018-12-29 15:42:17.157 -0.268  2.834  9.390
6       27 2018-12-21 11:34:13.653  0.919  1.149  9.686
7       27 2018-12-06 08:46:24.676  1.187  9.538  0.361
8       27 2018-12-12 12:30:08.205  2.260  8.657  3.556
9       27 2018-12-21 10:46:38.749  0.957  1.225  9.688


In [ ]:
st = time.time()
user46.to_pickle("cleaned_user46.pkl", compression="bz2")
et = time.time()
print("Pickling time:", et - st)


Pickling time: 85.67350268363953


In [ ]:
pred = pd.read_pickle("u46.pkl", compression="gzip")


In [ ]:
len(pred)


84341

In [ ]:
labels = pd.read_pickle("y_pred_u46.pkl", compression="gzip")


In [ ]:
pred.reset_index(drop=True, inplace=True)


In [ ]:
pred["activity_id"] = labels


In [ ]:
len(pred) - len(labels)


0

In [ ]:
pred


,user_id,start,finish,activity_id
0,27,2018-12-01 00:08:55.450,2018-12-01 00:41:33.580,5
1,27,2018-12-01 00:41:33.580,2018-12-01 01:12:50.040,5
2,27,2018-12-01 01:12:50.040,2018-12-01 01:24:52.758,5
3,27,2018-12-01 01:24:52.758,2018-12-01 01:33:04.510,5
4,27,2018-12-01 01:33:04.510,2018-12-01 01:39:05.730,5
...,...,...,...,...
84336,27,2019-01-01 09:13:14.000,2019-01-01 09:14:28.500,5
84337,27,2019-01-01 09:14:29.000,2019-01-01 09:16:20.500,5
84338,27,2019-01-01 09:35:48.500,2019-01-01 09:37:03.000,5
84339,27,2019-01-01 09:37:03.500,2019-01-01 09:38:18.000,5


In [ ]:
len(user46)


12539431

In [ ]:
pred.sort_values("start", inplace=True, ignore_index=True)
pred


,user_id,start,finish,activity_id
0,27,2018-12-01 00:08:55.000,2018-12-01 00:10:09.500,5
1,27,2018-12-01 00:08:55.450,2018-12-01 00:41:33.580,5
2,27,2018-12-01 00:10:10.000,2018-12-01 00:11:24.500,5
3,27,2018-12-01 00:11:25.000,2018-12-01 00:12:39.500,5
4,27,2018-12-01 00:12:40.000,2018-12-01 00:13:54.500,5
...,...,...,...,...
84336,27,2019-01-01 23:57:08.444,2019-01-01 23:57:39.117,5
84337,27,2019-01-01 23:57:50.997,2019-01-01 23:57:56.857,5
84338,27,2019-01-01 23:58:23.000,2019-01-01 23:59:59.500,5
84339,27,2019-01-01 23:58:37.663,2019-01-01 23:59:35.294,5


In [ ]:
user46["act_id"] = np.nan


In [ ]:
for i in range(len(pred)):
    matched = user46[
        (pred.loc[i, "start"] <= user46.datetime)
        & (pred.loc[i, "finish"] >= user46.datetime)
    ].index
    user46.loc[matched, "act_id"] = pred.loc[i, "activity_id"]
    # if not i % 500:
    #     print(i)


In [ ]:
user46["act_id"] = user46["act_id"].bfill()
user46["act_id"] = user46["act_id"].ffill()


In [ ]:
result46 = user46[["user_id", "datetime", "act_id"]]


In [ ]:
result46


,user_id,datetime,act_id
0,27,2018-12-11 16:46:05.707,4.0
1,27,2018-12-21 16:45:54.350,4.0
2,27,2018-12-21 10:38:58.230,5.0
3,27,2018-12-14 22:39:10.469,5.0
4,27,2018-12-07 15:43:06.481,5.0
...,...,...,...
12539426,27,2018-12-21 00:03:21.190,5.0
12539427,27,2018-12-21 02:27:55.440,4.0
12539428,27,2018-12-21 01:47:50.623,4.0
12539429,27,2018-12-21 00:31:11.222,5.0


In [ ]:
result46["act_id"] = result46["act_id"].astype(np.int8)
result46


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,user_id,datetime,act_id
0,27,2018-12-11 16:46:05.707,4
1,27,2018-12-21 16:45:54.350,4
2,27,2018-12-21 10:38:58.230,5
3,27,2018-12-14 22:39:10.469,5
4,27,2018-12-07 15:43:06.481,5
...,...,...,...
12539426,27,2018-12-21 00:03:21.190,5
12539427,27,2018-12-21 02:27:55.440,4
12539428,27,2018-12-21 01:47:50.623,4
12539429,27,2018-12-21 00:31:11.222,5


In [ ]:
result46.to_csv("result_user46.csv", index=False)


In [ ]:
result46.to_pickle("result_user46.pkl", compression="gzip")
